# Tunnel Score Exploratory Data Analysis

Jensen Holm
Sep. 2024

In [1]:
import polars as pl
import datetime
import pybaseball
from pybaseball import cache

from utils import pl_euclidean_distance

cache.enable()

orig_df = pl.from_pandas(
    pybaseball.statcast(
        start_dt="2024-03-01",
        end_dt=datetime.datetime.today().strftime("%Y-%m-%d"),
    )
)

orig_df.sample(3, seed=1)

This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 174/174 [01:02<00:00,  2.78it/s]
/Users/jensen/Documents/projects/TunnelScoreEDA/venv/lib/python3.12/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,…,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
str,datetime[ns],f64,f64,f64,str,i64,i64,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,i64,str,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,str,…,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,f64,f64
"""SI""",2024-05-05 00:00:00,90.6,-1.24,5.14,"""Webb, Logan""",592206,657277,"""single""","""hit_into_play""",null,null,null,null,7,"""Nick Castellanos singles on a …","""R""","""R""","""R""","""PHI""","""SF""","""X""",7,"""line_drive""",0,0,2024,-1.05,0.24,-0.64,2.19,null,null,null,2,3,"""Bot""",…,6.9,745587,657277,666165,664774,642731,656305,605113,624424,808982,573262,53.6,0.837,0.821,0.9,1,1,0,5,27,1,"""Sinker""",4,1,4,1,1,4,4,1,"""Standard""","""Standard""",237,0.006,0.13,null,null
"""CH""",2024-08-28 00:00:00,79.0,-1.71,6.73,"""Wacha, Michael""",686823,608379,"""field_out""","""hit_into_play""",null,null,null,null,13,"""Will Brennan lines out to cent…","""R""","""L""","""R""","""CLE""","""KC""","""X""",8,"""line_drive""",0,1,2024,-1.43,0.0,-1.24,2.12,null,null,null,2,4,"""Bot""",…,6.6,746589,608379,666023,686469,686681,657557,677951,669004,664728,624428,53.86,0.323,0.291,0.0,1,0,0,3,33,2,"""Changeup""",1,3,1,3,3,1,1,3,"""Standard""","""Standard""",250,-0.013,-0.085,62.40505,7.35537
"""SL""",2024-07-06 00:00:00,83.1,1.12,5.8,"""Falter, Bailey""",621438,663559,null,"""ball""",null,null,null,null,11,"""Tyrone Taylor doubles (10) on …","""R""","""R""","""L""","""PIT""","""NYM""","""B""",null,null,1,1,2024,-0.51,0.21,-0.4,4.11,null,null,620443,0,2,"""Top""",…,7.1,745476,663559,663698,656582,693304,663647,665833,668804,572191,641943,53.36,null,null,null,null,null,null,null,10,3,"""Slider""",0,0,0,0,0,0,0,0,"""Standard""","""Standard""",218,0.0,0.066,null,null


## Tying pitches to their previous ones

This will allow us to compare each pitch to the one before it, giving us the ability to calculate tunnel score.

In [2]:
# first we need to sort all the pitches

SORT_COLS = [
    "game_date",
    "pitcher",
    "at_bat_number",
    "pitch_number",
]

shifted_df = orig_df.sort(SORT_COLS, descending=True)

OVER_COLS = ["at_bat_number", "pitcher"]
# now we can shift the data by one row to get the previous pitch data for each pitch in one row
for col_name in shifted_df.columns:
    shifted_df = shifted_df.with_columns(
        pl.col(col_name).shift(-1).over(OVER_COLS).alias(f"prev_{col_name}")
    )

shifted_df.select(SORT_COLS).sample(5, seed=1)

game_date,pitcher,at_bat_number,pitch_number
datetime[ns],i64,i64,i64
2024-05-05 00:00:00,641482,8,6
2024-08-28 00:00:00,674370,49,2
2024-08-05 00:00:00,605400,36,2
2024-07-06 00:00:00,608344,63,3
2024-06-14 00:00:00,640455,10,2


## Adding Tunnel Score

```math
TunnelScore = log_{2}( (\frac{TunnelDistance}{ActualDistance}) - ReleaseDistance)
```

Where ...

**ReleaseDistance** = euclidean distance between the release points of current pitch and the previous pitch

**ActualDistance** = euclidean distance between where the current pitch ended up over the plate, and the previous pitch 

**TunnelDistance** = euclidean distance between where the current pitch would have ended up without movement, and the same for the previous pitch 


In [3]:
# compute where the pitch and the previous one would have been if they had no movement
tunnel_distance_df: pl.DataFrame = shifted_df.with_columns(
    plate_x_no_move=pl.col("plate_x")
    - pl.col("pfx_x"),  # x coord of the pitch if it had no movement
    plate_z_no_move=pl.col("plate_z")
    - pl.col("pfx_z"),  # z coord of the pitch if it had no movement
    prev_plate_x_no_move=pl.col("prev_plate_x")
    - pl.col("prev_pfx_x"),  # x coord of the previous pitch if it had no movement
    prev_plate_z_no_move=pl.col("prev_plate_z")
    - pl.col("prev_pfx_z"),  # z coord of the previous pitch if it had no movement
)

tunnel_cols_df: pl.DataFrame = tunnel_distance_df.with_columns(
    # tunnel_distance is the distance between the pitch and the previous pitch if they had no movement
    tunnel_distance=pl_euclidean_distance(
        x1=pl.col("plate_x_no_move"),
        y1=pl.col("plate_z_no_move"),
        x2=pl.col("prev_plate_x_no_move"),
        y2=pl.col("prev_plate_z_no_move"),
    ),
    # actual_distance is the distance between the pitch and the previous pitch
    actual_distance=pl_euclidean_distance(
        x1=pl.col("plate_x"),
        y1=pl.col("plate_z"),
        x2=pl.col("prev_plate_x"),
        y2=pl.col("prev_plate_z"),
    ),
    # release_distance is the distance between the release point of the pitch and the previous pitch
    release_distance=pl_euclidean_distance(
        x1=pl.col("release_pos_x"),
        y1=pl.col("release_pos_z"),
        x2=pl.col("prev_release_pos_x"),
        y2=pl.col("prev_release_pos_z"),
    ),
)

TUNNEL_COLS = [
    "tunnel_distance",
    "actual_distance",
    "release_distance",
]

tunnel_cols_df.select(SORT_COLS + TUNNEL_COLS).sample(5, seed=1)

game_date,pitcher,at_bat_number,pitch_number,tunnel_distance,actual_distance,release_distance
datetime[ns],i64,i64,i64,f64,f64,f64
2024-05-05 00:00:00,641482,8,6,0.6776,4.8204,4.9825
2024-08-28 00:00:00,674370,49,2,4.9544,5.5,0.0309
2024-08-05 00:00:00,605400,36,2,1.2425,1.4825,0.0209
2024-07-06 00:00:00,608344,63,3,1.8396,2.2741,0.17
2024-06-14 00:00:00,640455,10,2,0.79,0.6729,0.0556


In [4]:
# now we can add tunnel score column using the tunnel_distance, actual_distance, and release_distance

# adding this if statement to avoid adding the same column multiple times
# because the nature of jupyter notebooks
if "tunnel_score" not in TUNNEL_COLS:
    TUNNEL_COLS.append("tunnel_score")

tunnel_score_df = tunnel_cols_df.with_columns(
    tunnel_score=(pl.col("tunnel_distance") / pl.col("actual_distance"))
    - pl.col("release_distance").log(base=2),
).drop_nulls(subset=TUNNEL_COLS)

print(f"shape of dataframe with tunnel score: {tunnel_score_df.shape}")
tunnel_score_df.select(SORT_COLS + TUNNEL_COLS + ["player_name"])

shape of dataframe with tunnel score: (592197, 196)


game_date,pitcher,at_bat_number,pitch_number,tunnel_distance,actual_distance,release_distance,tunnel_score,player_name
datetime[ns],i64,i64,i64,f64,f64,f64,f64,str
2024-09-03 00:00:00,701581,42,7,0.0725,1.8141,0.3061,1.74789,"""Festa, David"""
2024-09-03 00:00:00,701581,42,6,3.7461,3.7076,0.2744,2.876032,"""Festa, David"""
2024-09-03 00:00:00,701581,42,5,2.3169,3.3069,0.1816,3.16179,"""Festa, David"""
2024-09-03 00:00:00,701581,42,4,0.6301,0.5904,0.0516,5.343728,"""Festa, David"""
2024-09-03 00:00:00,701581,42,3,3.0436,3.19,0.15,3.691072,"""Festa, David"""
…,…,…,…,…,…,…,…,…
2024-03-15 00:00:00,516853,78,2,2.79,1.6541,0.2524,3.672934,"""Rodríguez, Wilking"""
2024-03-15 00:00:00,516853,77,5,0.7269,3.1725,0.1549,2.919716,"""Rodríguez, Wilking"""
2024-03-15 00:00:00,516853,77,4,1.3629,1.8276,0.1241,3.756157,"""Rodríguez, Wilking"""


## Explore Tunnel Score relationships on AB level

Interested to see if there is a strong relationship between AB's that had at least one pitch with high tunnel score, and getting an out.

## Joining with FanGraphs data

I am interested to see if at bats with a high avg, median, max, tunnel score have a relationship with good results for the pitcher. Currently the data we have is just statcast pitch level data, so I am going to join it with season summary data from fangraphs and aggregate tunnel score metrics.

In [5]:
# first loading other player data to join with statcast data
fangraphs_pitching_df = pybaseball.pitching_stats(2024)

player_fg_ids_df = pybaseball.playerid_reverse_lookup(
    fangraphs_pitching_df["IDfg"].unique().tolist(), key_type="fangraphs"
)

player_ids_df = pl.from_pandas(fangraphs_pitching_df).join(
    other=pl.from_pandas(player_fg_ids_df),
    left_on="IDfg",
    right_on="key_fangraphs",
    how="left",
    validate="1:1",
)

player_ids_df.sample(5, seed=1)

IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,…,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO,name_last,name_first,key_mlbam,key_retro,key_bbref,mlb_played_first,mlb_played_last
i64,i64,str,str,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,str,str,i64,str,str,f64,f64
9803,2024,"""Miles Mikolas""","""STL""",35,8,10,1.9,5.27,28,28,0,0,0,0,153.2,642,171,93,90,22,24,0,3,0,1,107,216,181,108,19,770,1636,2406,73,15,2,…,86.0,103.0,98.0,96.0,108.0,105.0,66.0,107.0,97.0,83.0,110.0,98.0,116.0,111.0,114.0,null,null,null,null,null,null,null,null,null,89,108,102,null,null,null,"""mikolas""","""miles""",571945,"""mikom001""","""mikolmi01""",2012.0,2024.0
10310,2024,"""Zack Wheeler""","""PHI""",34,13,6,4.4,2.63,27,27,0,0,0,0,167.2,665,119,54,49,17,45,0,8,8,0,183,180,167,80,25,924,1735,2659,96,11,2,…,null,null,null,111.0,101.0,107.0,100.0,107.0,110.0,111.0,103.0,108.0,128.0,107.0,116.0,null,null,null,92.0,97.0,95.0,95.0,107.0,107.0,105,105,108,null,null,null,"""wheeler""","""zack""",554430,"""wheez001""","""wheelza01""",2013.0,2024.0
19879,2024,"""Tanner Houck""","""BOS""",28,8,9,3.6,3.12,27,27,1,1,0,0,164.2,689,146,70,57,10,46,1,15,2,0,143,268,116,98,6,857,1678,2535,101,14,2,…,null,null,null,null,null,null,null,null,null,93.0,102.0,96.0,117.0,98.0,109.0,null,null,null,95.0,93.0,93.0,116.0,103.0,110.0,109,101,105,null,null,null,"""houck""","""tanner""",656557,"""houct001""","""houckta01""",2020.0,2024.0
16149,2024,"""Aaron Nola""","""PHI""",31,12,6,2.9,3.29,28,28,1,1,0,0,172.1,700,155,67,63,24,43,0,3,2,0,162,215,169,107,14,929,1778,2707,109,10,1,…,79.0,102.0,90.0,null,null,null,90.0,103.0,99.0,100.0,111.0,108.0,null,null,null,135.0,107.0,114.0,69.0,98.0,92.0,null,null,null,104,105,104,null,null,null,"""nola""","""aaron""",605400,"""nolaa001""","""nolaaa01""",2015.0,2024.0
9132,2024,"""Nathan Eovaldi""","""TEX""",34,10,7,2.6,3.6,24,24,0,0,0,0,140.0,564,117,57,56,18,32,0,6,3,0,138,191,119,76,9,693,1463,2156,75,6,2,…,null,null,null,101.0,103.0,104.0,75.0,106.0,99.0,null,null,null,115.0,100.0,103.0,null,null,null,91.0,107.0,98.0,127.0,102.0,111.0,98,105,104,null,null,null,"""eovaldi""","""nathan""",543135,"""eovan001""","""eovalna01""",2011.0,2024.0


In [6]:
# NOTE: this drops some data for newer players that are not in the playerid_reverse_lookup
# (you can see this in the difference of the shapes of this dataframe and tunnel_score_df)
tunnel_score_fg_df = tunnel_score_df.join(
    other=player_ids_df,
    left_on="pitcher",
    right_on="key_mlbam",
    how="inner",  # getting rid of players that are not in the fangraphs_pitching_df
    validate="m:1",
)

print(f"tunnel score fangraphs df shape: {tunnel_score_fg_df.shape}")
tunnel_score_fg_df.select(
    SORT_COLS + TUNNEL_COLS + ["player_name", "BB%", "K%"]
).sample(5, seed=1)

tunnel score fangraphs df shape: (157212, 595)


game_date,pitcher,at_bat_number,pitch_number,tunnel_distance,actual_distance,release_distance,tunnel_score,player_name,BB%,K%
datetime[ns],i64,i64,i64,f64,f64,f64,f64,str,f64,f64
2024-05-08 00:00:00,669923,32,3,1.4241,8.2481,0.0981,3.522261,"""Kirby, George""",0.032,0.23
2024-08-27 00:00:00,694297,10,4,2.6836,0.7544,0.3256,5.176091,"""Pfaadt, Brandon""",0.051,0.233
2024-07-06 00:00:00,642547,12,1,1.0484,1.1744,0.2096,3.147001,"""Peralta, Freddy""",0.091,0.273
2024-06-25 00:00:00,686613,44,6,0.4721,2.2901,0.0356,5.018127,"""Brown, Hunter""",0.088,0.252
2024-07-14 00:00:00,669854,10,6,2.3861,2.7824,0.4144,2.128473,"""Blanco, Ronel""",0.101,0.24
